In [1]:
from Selector import *
import sys
import time
from view.HTML import show
import json
from indicators.MA import *
from simulation.Simulator import *
from indicators.Trend import *
sys.path.append("../data")
import KDataDao as kds
from view.ViewDataFactory import *
from strategy.sequence.StrategyS1 import *
from simulation.SequenceSimulatorS import *
from simulation.ParallelSimulator import *
from strategy.sequence.MAStrategyS import *
from strategy.sequence.RStrategy import *
from indicators.TrendArea import *
from strategy.parallel.StrategyP import *
from strategy.parallel.BuyA import *
from strategy.parallel.SellA import *
from strategy.parallel.SellB import *
from strategy.parallel.SellS import *
from strategy.component.ConditionBean import *

In [2]:
class KStrategy:
    """
    K线策略
    """
    def __init__(self,trader=None,listK=None):
        self.point = 0 
        self.trend = TrendArea()
        self.condition = None
        self.listK = None
        self.trader = None
    def init(self,trader,listK):
        self.trader = trader
        self.listK = listK
        self.trend = TrendArea()
    def decide(self,i):
        k = self.listK[i]
        if self.trader.isHold(k.code):
            if k.isDown():
                self.trader.sell(k.code, k.date, k.close, condition="k1")
        else:
            if k.isUp():
                self.trader.buy(k.code, k.date, k.close, condition="k1")

In [3]:
#测试策略
# bDate = "2008-06-05"
bDate = "2015-03-05"
eDate = "2016-11-07"
market = "sza"
begin = time.time()
print('runing......')
trader = Trader()
buys = [BuyA()]
sells = [SellA("a"),SellB('a'),SellS("a")]
s1 = StrategyP(trader,buys,sells)
# s1 = MAStrategyS(30)
sim1 = ParallelSimulator(s1)
sim1.run(market,bDate,eDate)
print('complete %s second' %  (time.time()-begin))

runing......
complete 1.019528865814209 second


In [4]:
# begin = time.time()
# print 'runing......'
# s12 = KStrategy2()
# sim12 = SequenceSimulator(s12)
# sim12.run(market,bDate,eDate,threadCount=0)
# print 'complete %s second' %  (time.time()-begin)

In [5]:
#均线策略
begin = time.time()
print('runing......')
s2 = MAStrategyS(60)
sim2 = SequenceSimulatorS(s2)
sim2.run(market,bDate,eDate,threadCount=0)
print('complete %s second' %  (time.time()-begin))

runing......
thread-main begin run
thread-main end run
complete 27.306427001953125 second


In [6]:
#K线策略
begin = time.time()
print('runing......')
ks = KStrategy()
simK = SequenceSimulatorS(ks)
simK.run(market,bDate,eDate,threadCount=0)
print('complete %s second' %  (time.time()-begin))

runing......
thread-main begin run


TypeError: list indices must be integers or slices, not K

In [ ]:
#择时收益
sim3 = Simulator(RStrategy())
sim3.runSingle('zs_000001',begin=bDate,end=eDate)

In [ ]:
listK = kd.get_k_data('zs_000001',bDate,eDate)
data,volume = ViewDataFactory.getCandlestick(listK)

In [ ]:
avgEarningsLine = sim1.trader.earningsLine

oddsLine,win,lose,odds = sim2.getOdds()
# avgEarningsLine2,maxCode2,maxEarningsLine2,minCode2,minEarningsLine2 = sim12.earningsLines()
maEarningsLine,maxCode,maxEarningsLine,minCode,minEarningsLine = sim2.earningsLines()
kEarningsLine,kMaxCode,kMaxEarningsLine,kMinCode,kMinEarningsLine = simK.earningsLines()

rEarningsLine = sim3.trader.earningsLine


params = {}
params["maxCode"] = maxCode
params["minCode"] = minCode
params["avgEarningsLine"] = json.dumps(avgEarningsLine)
params["avgEarningsLine2"] = {}#json.dumps(avgEarningsLine2)
params["maxEarningsLine"] = json.dumps(maxEarningsLine)
params["minEarningsLine"] = json.dumps(minEarningsLine)
params["maEarningsLine"] = json.dumps(maEarningsLine)
params["rEarningsLine"] = json.dumps(rEarningsLine)
params["kEarningsLine"] = json.dumps(kEarningsLine)
params["oddsLine"] = json.dumps(oddsLine)
params["win"] = json.dumps(win)
params["lose"] = json.dumps(lose)
params["odds"] = json.dumps(odds)
params["data"] = json.dumps(data)
params["volume"] = json.dumps(volume)
clear_output()
s = show("Strategy.html",params)
